In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import table
import os, sys, json
from pulp import *

%matplotlib inline

In [2]:
#Create the LPProb for the 15-min interval schedule
def LpProb(periods, agents_per_shift):
    
    ##Generating variables for every period type, in a shift
    
    #short break
    s = {
     (p, n): pulp.LpVariable(
        "Small_break_{0}_{1}".format(p, n), cat = "Binary")
        for p in range(periods) for n in range(agents_per_shift)
    }
    
    #on queue
    w = {
     (p, n): pulp.LpVariable(
        "Working_period_{0}_{1}".format(p, n), cat = "Binary")
        for p in range(periods) for n in range(agents_per_shift)
    }
    
    #meal break
    l = {
     (p, n): pulp.LpVariable(
        "Long_break_{0}_{1}".format(p, n), cat = "Binary")
        for p in range(periods) for n in range(agents_per_shift)
    }
    
    #training
    t = {
     (p, n): pulp.LpVariable(
        "Training_Period_{0}_{1}".format(p, n), cat = "Binary")
        for p in range(periods) for n in range(agents_per_shift)
    }
    
    problem = LpProblem("Shift",LpMinimize)
    
    # # The prob variable is created to contain the problem data   
    obj = None
    for p in range(periods):
        for n in range(agents_per_shift):
            obj += w[(p,n)] + s[(p,n)] + l[(p,n)] + t[(p,n)]
    problem += obj
            

    # add constraints: 
    # 1 : At any given period, s + w + l + t = 1
    for p in range(periods):
        for n in range(agents_per_shift):
            problem.addConstraint(w[(p,n)] + s[(p,n)] + l[(p,n)] + t[(p,n)]== 1)
            
    # constraint 2 : minimum agents per period = 2 (VARIABLE)
    for p in range(0,periods):
        d = None
        for n in range(agents_per_shift):
            d+= w[(p,n)] 
        problem += d >= 2
        
        
    # constraint 3: w should be 11 working periods      
    for n in range(agents_per_shift):
        problem.addConstraint(
            sum(w[(p,n)] for p in range(periods)) == 11
        )
    
    for n in range(agents_per_shift):
        problem.addConstraint(
            sum(s[(p,n)] for p in range(1, periods)) == 1
        )
        
    for n in range(agents_per_shift):
        problem.addConstraint(
            sum(l[(p,n)] for p in range(1, periods)) == 2
        )
        
    for n in range(agents_per_shift):
        problem.addConstraint(
            sum(t[(p,n)] for p in range(1, periods)) == 2
        )
     
    # constraint 4: consecutive 2 long break periods of 1 hour (1 period = 30 mins)
    for n in range(agents_per_shift):
        for p in range(0,periods-2):
            g = None
            g += l[(p,n)] - l[(p+1,n)] + l[(p+2,n)]
            problem += g >= 0
            
            
   # constraint 5: consecutive 2 training periods of 1 hour (1 period = 30 mins)
    for n in range(agents_per_shift):
        for p in range(0,periods-2):
            h = None
            h += t[(p,n)] - t[(p+1,n)] + t[(p+2,n)]
            problem += h >= 0
    
    
    #constraint 6: long break and short break cannot be together
    for n in range(agents_per_shift):
        for p in range(0,periods-1):
            a = None
            b = None
            a += l[(p,n)] + s[(p+1,n)]
            b += s[(p,n)] + l[(p+1,n)]
            problem += a <= 1
            problem += b <= 1
        
    return s,w,l,t, problem

In [3]:
def agent_scheduling(on_queue, meal_break, short_break, training, period_name, agent_id_working_in_shift):


    schedule = pd.DataFrame(data=None, index = period_name, columns = agent_id_working_in_shift)
    for k, v in on_queue.items():
        p, n = k[0], k[1]
        current_index = p*2
        if(int(value(v)) == 1):
            schedule.iloc[current_index][n] = "On_queue" # pulp.value()
            current_index+=1
            schedule.iloc[current_index][n] = "On_queue"
        else:
            schedule.iloc[current_index][n] = "" # pulp.value()
            current_index+=1
            schedule.iloc[current_index][n] = ""
    
    
    for k, v in meal_break.items():
        p, n = k[0], k[1]
        current_index = p*2
        if(int(value(v)) == 1):
            schedule.iloc[current_index][n] = "Meal_break" # pulp.value()
            current_index+=1
            schedule.iloc[current_index][n] = "Meal_break"
    
    for k, v in short_break.items():
        p, n = k[0], k[1]
        current_index = p*2
        if(int(value(v)) == 1):
            schedule.iloc[current_index][n] = "Short_break" # pulp.value()
            current_index+=1
            schedule.iloc[current_index][n] = "Short_break"
    
    for k, v in training.items():
        p, n = k[0], k[1]
        current_index = p*2
        if(int(value(v)) == 1):
            schedule.iloc[current_index][n] = "Training" # pulp.value()
            current_index+=1
            schedule.iloc[current_index][n] = "Training"
    
    return schedule

In [4]:
# def color_rule(val):
#     color = 'white'
# #     for string in cell_values:
#     if val == 'Off':
#         color = 'grey'
#     elif val == "On_queue":
#         color = 'green'
#     elif val == "Meal_break":
#         color = 'red'
#     elif val == "Short_break":
#         color = 'blue'
#     elif val == "Training":
#         color = 'brown'
#     else:
#         color = 'orange'
    
#     return 'color: %s' % color

In [5]:
def schedule_daily_shift(agents, shift, shift_name, all_agents_id, predicted_agents_per_period):
    
    periods = 1*16
    agents_per_shift = len(agents)

    ## Create shift names based on index:
    period_name = []

    period_1 = ["00:00-00:15","00:15-00:30","00:30-00:45","00:45-1:00","1:00-1:15", "1:15-1:30", "1:30-1:45", "1:45-2:00", "2:00-2:15", "2:15-2:30", "2:30-2:45", "2:45-3:00", "3:00-3:15", "3:15-3:30", "3:30-3:45", "3:45-4:00", "4:00-4:15", "4:15-4:30", "4:30-4:45", "4:45-5:00", "5:00-5:15", "5:15-5:30", "5:30-5:45", "5:45-6:00", "6:00-6:15", "6:15-6:30", "6:30-6:45", "6:45-7:00", "7:00-7:15", "7:15-7:30", "7:30-7:45", "7:45-8:00"]
    period_2 = ["8:00-8:15","8:15-8:30","8:30-8:45","8:45-9:00","9:00-9:15", "9:15-9:30", "9:30-9:45", "9:45-10:00", "10:00-10:15", "10:15-10:30", "10:30-10:45", "10:45-11:00", "11:00-11:15", "11:15-11:30", "11:30-11:45", "11:45-12:00", "12:00-12:15", "12:15-12:30", "12:30-12:45", "12:45-13:00", "13:00-13:15", "13:15-13:30", "13:30-13:45", "13:45-14:00", "14:00-14:15", "14:15-14:30", "14:30-14:45", "14:45-15:00", "15:00-15:15", "15:15-15:30", "15:30-15:45", "15:45-16:00"]
    period_3 = ["16:00-16:15","16:15-16:30","16:30-16:45","16:45-17:00","17:00-17:15", "17:15-17:30", "17:30-17:45", "17:45-18:00", "18:00-18:15", "18:15-18:30", "18:30-18:45", "18:45-19:00", "19:00-19:15", "19:15-19:30", "19:30-19:45", "19:45-20:00", "20:00-20:15", "20:15-20:30", "20:30-20:45", "20:45-21:00", "21:00-21:15", "21:15-21:30", "21:30-21:45", "21:45-22:00", "22:00-22:15", "22:15-22:30", "22:30-22:45", "22:45-23:00", "23:00-23:15", "23:15-23:30", "23:30-23:45", "23:45-00:00"]

    if shift%3 == 0:
        period_name = period_1
    elif shift%3 == 1:
        period_name = period_2
    else:
        period_name = period_3
        
    
    
    ## Run the solver
    LpSolverDefault.msg = 1
    s,w,l,t, problem = LpProb(periods, agents_per_shift)
    problem.solve()
    print(LpStatus[problem.status])
    
    
    schedule = agent_scheduling(w, l, s, t, period_name, agents)
    for i, row in schedule.iterrows():
        agents_on_queue = 0
        for j, column in row.iteritems():
            if column == "On_queue":
                agents_on_queue+=1
        predicted_agents_per_period.append(agents_on_queue)
        
    for agent_id in all_agents_id:
        if agent_id not in agents:
            schedule[agent_id] = 'Off'
    
    schedule['Shift'] = shift_name
    print(schedule)
    
#     final_schedule = schedule.style.applymap(color_rule)

    if shift == 0:
        schedule.to_csv('agent_wise_shift_wise_schedule.csv', mode='w')
    else:
        schedule.to_csv('agent_wise_shift_wise_schedule.csv', mode='a')
        
        
#     if shift == 0:
#         sheetName = "Day " + str(shift)
#         with pd.ExcelWriter('agent_wise_shift_wise_schedule.xlsx', engine='openpyxl', mode="w") as f:
#             final_schedule.to_excel(f, sheet_name=sheetName)
#     else:
#         sheetName = "Day " + str(shift)
#         with pd.ExcelWriter('agent_wise_shift_wise_schedule.xlsx',  engine='openpyxl', mode="a") as f:
#             final_schedule.to_excel(f, sheet_name=sheetName)    

In [6]:
#Calculate delta (On_Queue - Forecasted)

def desired_output(predicted_agents_per_period):
    delta_df = pd.read_excel("C:/Users/19084/Downloads/mocked_scheduler(Input for forecast).xlsx", sheet_name='forecast')
    delta_df = delta_df.dropna(how='all', axis='columns')
    delta_df['On_Queue'] = predicted_agents_per_period
    delta_df['Delta, i.e. On_Queue - Forcasted'] = delta_df['On_Queue'].sub(delta_df['Forecasted'], axis = 0)
    delta_df = delta_df[['Day & Time','Forecasted', 'On_Queue', 'Delta, i.e. On_Queue - Forcasted']]
    delta_df.to_csv('delta.csv', mode='w')
    print(delta_df)

In [7]:


shift_index = 0
%store -r shift_name_map
%store -r agent_id

predicted_agents_per_period = []
for shift_name, agents in shift_name_map.items():
    print("Shift Name: ", shift_name)
    schedule_daily_shift(agents, shift_index, shift_name, agent_id, predicted_agents_per_period)
    shift_index+=1
    #     input("Press Enter to continue...")
    
desired_output(predicted_agents_per_period)

Shift Name:  M_s1
Optimal
                  agent7      agent10      agent13      agent18 agent0 agent1  \
00:00-00:15     On_queue     On_queue     On_queue     On_queue    Off    Off   
00:15-00:30     On_queue     On_queue     On_queue     On_queue    Off    Off   
00:30-00:45     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
00:45-1:00      On_queue   Meal_break     On_queue   Meal_break    Off    Off   
1:00-1:15       On_queue   Meal_break     On_queue   Meal_break    Off    Off   
1:15-1:30       On_queue   Meal_break     On_queue   Meal_break    Off    Off   
1:30-1:45       On_queue     On_queue   Meal_break     On_queue    Off    Off   
1:45-2:00       On_queue     On_queue   Meal_break     On_queue    Off    Off   
2:00-2:15       On_queue     On_queue   Meal_break     On_queue    Off    Off   
2:15-2:30       On_queue     On_queue   Meal_break     On_queue    Off    Off   
2:30-2:45       On_queue     Training     On_queue     On_queue    Off    Off   
2:

Optimal
                  agent2       agent5      agent11      agent15 agent0 agent1  \
16:00-16:15     On_queue     On_queue     On_queue     On_queue    Off    Off   
16:15-16:30     On_queue     On_queue     On_queue     On_queue    Off    Off   
16:30-16:45     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
16:45-17:00     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
17:00-17:15     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
17:15-17:30     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
17:30-17:45     On_queue     On_queue   Meal_break     On_queue    Off    Off   
17:45-18:00     On_queue     On_queue   Meal_break     On_queue    Off    Off   
18:00-18:15     On_queue     On_queue   Meal_break     On_queue    Off    Off   
18:15-18:30     On_queue     On_queue   Meal_break     On_queue    Off    Off   
18:30-18:45     On_queue     Training     On_queue     On_queue    Off    Off   
18:45-19:00     On_q

[32 rows x 21 columns]
Shift Name:  Tu_s3
Optimal
                  agent0       agent6      agent15      agent16 agent1 agent2  \
16:00-16:15     On_queue     On_queue     On_queue     On_queue    Off    Off   
16:15-16:30     On_queue     On_queue     On_queue     On_queue    Off    Off   
16:30-16:45     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
16:45-17:00     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
17:00-17:15     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
17:15-17:30     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
17:30-17:45     On_queue     On_queue   Meal_break     On_queue    Off    Off   
17:45-18:00     On_queue     On_queue   Meal_break     On_queue    Off    Off   
18:00-18:15     On_queue     On_queue   Meal_break     On_queue    Off    Off   
18:15-18:30     On_queue     On_queue   Meal_break     On_queue    Off    Off   
18:30-18:45     On_queue     Training     On_queue     On_q

[32 rows x 21 columns]
Shift Name:  Th_s1
Optimal
                  agent1       agent4       agent9      agent16 agent0 agent2  \
00:00-00:15     On_queue     On_queue     On_queue     On_queue    Off    Off   
00:15-00:30     On_queue     On_queue     On_queue     On_queue    Off    Off   
00:30-00:45     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
00:45-1:00      On_queue   Meal_break     On_queue   Meal_break    Off    Off   
1:00-1:15       On_queue   Meal_break     On_queue   Meal_break    Off    Off   
1:15-1:30       On_queue   Meal_break     On_queue   Meal_break    Off    Off   
1:30-1:45       On_queue     On_queue   Meal_break     On_queue    Off    Off   
1:45-2:00       On_queue     On_queue   Meal_break     On_queue    Off    Off   
2:00-2:15       On_queue     On_queue   Meal_break     On_queue    Off    Off   
2:15-2:30       On_queue     On_queue   Meal_break     On_queue    Off    Off   
2:30-2:45       On_queue     Training     On_queue     On_q

Shift Name:  F_s2
Optimal
                  agent9      agent14      agent15      agent17 agent0 agent1  \
8:00-8:15       On_queue     On_queue     On_queue     On_queue    Off    Off   
8:15-8:30       On_queue     On_queue     On_queue     On_queue    Off    Off   
8:30-8:45       On_queue   Meal_break     On_queue   Meal_break    Off    Off   
8:45-9:00       On_queue   Meal_break     On_queue   Meal_break    Off    Off   
9:00-9:15       On_queue   Meal_break     On_queue   Meal_break    Off    Off   
9:15-9:30       On_queue   Meal_break     On_queue   Meal_break    Off    Off   
9:30-9:45       On_queue     On_queue   Meal_break     On_queue    Off    Off   
9:45-10:00      On_queue     On_queue   Meal_break     On_queue    Off    Off   
10:00-10:15     On_queue     On_queue   Meal_break     On_queue    Off    Off   
10:15-10:30     On_queue     On_queue   Meal_break     On_queue    Off    Off   
10:30-10:45     On_queue     Training     On_queue     On_queue    Off    Off   
10

Optimal
                  agent1       agent3       agent6      agent18 agent0 agent2  \
00:00-00:15     On_queue     On_queue     On_queue     On_queue    Off    Off   
00:15-00:30     On_queue     On_queue     On_queue     On_queue    Off    Off   
00:30-00:45     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
00:45-1:00      On_queue   Meal_break     On_queue   Meal_break    Off    Off   
1:00-1:15       On_queue   Meal_break     On_queue   Meal_break    Off    Off   
1:15-1:30       On_queue   Meal_break     On_queue   Meal_break    Off    Off   
1:30-1:45       On_queue     On_queue   Meal_break     On_queue    Off    Off   
1:45-2:00       On_queue     On_queue   Meal_break     On_queue    Off    Off   
2:00-2:15       On_queue     On_queue   Meal_break     On_queue    Off    Off   
2:15-2:30       On_queue     On_queue   Meal_break     On_queue    Off    Off   
2:30-2:45       On_queue     Training     On_queue     On_queue    Off    Off   
2:45-3:00       On_q

Shift Name:  Su_s1
Optimal
                  agent3       agent6       agent9      agent16 agent0 agent1  \
00:00-00:15     On_queue     On_queue     On_queue     On_queue    Off    Off   
00:15-00:30     On_queue     On_queue     On_queue     On_queue    Off    Off   
00:30-00:45     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
00:45-1:00      On_queue   Meal_break     On_queue   Meal_break    Off    Off   
1:00-1:15       On_queue   Meal_break     On_queue   Meal_break    Off    Off   
1:15-1:30       On_queue   Meal_break     On_queue   Meal_break    Off    Off   
1:30-1:45       On_queue     On_queue   Meal_break     On_queue    Off    Off   
1:45-2:00       On_queue     On_queue   Meal_break     On_queue    Off    Off   
2:00-2:15       On_queue     On_queue   Meal_break     On_queue    Off    Off   
2:15-2:30       On_queue     On_queue   Meal_break     On_queue    Off    Off   
2:30-2:45       On_queue     Training     On_queue     On_queue    Off    Off   
2

Optimal
                  agent0      agent10      agent11      agent13 agent1 agent2  \
16:00-16:15     On_queue     On_queue     On_queue     On_queue    Off    Off   
16:15-16:30     On_queue     On_queue     On_queue     On_queue    Off    Off   
16:30-16:45     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
16:45-17:00     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
17:00-17:15     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
17:15-17:30     On_queue   Meal_break     On_queue   Meal_break    Off    Off   
17:30-17:45     On_queue     On_queue   Meal_break     On_queue    Off    Off   
17:45-18:00     On_queue     On_queue   Meal_break     On_queue    Off    Off   
18:00-18:15     On_queue     On_queue   Meal_break     On_queue    Off    Off   
18:15-18:30     On_queue     On_queue   Meal_break     On_queue    Off    Off   
18:30-18:45     On_queue     Training     On_queue     On_queue    Off    Off   
18:45-19:00     On_q